In [26]:
"""
An example interface designed to be imported in your projects as a library.
"""
import urllib.request
import ssl
import json
from typing import List, Dict, Any

class APIError(Exception):
    def __init__(self, message: str, correlation_id: str = None):
        self.message = message
        self.correlation_id = correlation_id
        super().__init__(message)


class FinancialDataAPI:
    def __init__(self, certificate_path: str):
        self.url = 'https://web.api.six-group.com/api/findata'
        self.headers = {
            "accept": "application/json"
        }
        self.context = ssl.SSLContext()
        self.context.load_cert_chain(f'{certificate_path}/signed-certificate.pem', f'{certificate_path}/private-key.pem')

    def _http_request(self, end_point: str, query_string: Dict[str, Any]) -> Dict[str, Any]:
        """
        Make an HTTP request and send the raw response.
        """
        complete_url = f"{self.url}{end_point}?{urllib.parse.urlencode(query_string)}"
        try:
            request = urllib.request.Request(complete_url, headers=self.headers)
            with urllib.request.urlopen(request, context=self.context) as response:
                return json.loads(response.read())
        except urllib.error.HTTPError as err:
            correlation_id = err.headers.get('X-CorrelationID')
            raise APIError("An error occurred during the API request.", correlation_id) from err

    def _http_request_with_scheme_id(self, end_point: str, scheme: str, ids: List[str]) -> Dict[str, Any]:
        """
        Make an HTTP request using scheme and ids.
        """
        query_string = {
            'scheme': scheme,
            'ids': ",".join(ids)
        }
        return self._http_request(end_point, query_string)

    def instrumentBase(self, scheme: str, instruments: List[str]) -> Dict[str, Any]:
        """
        Retrieve instrument basic attributes using scheme and ids.
        """
        end_point = "/v1/instruments/referenceData/instrumentBase"
        return self._http_request_with_scheme_id(end_point, scheme, instruments)

    def endOfDayHistory(self, scheme: str, listings: List[str], dateFrom: str, dateTo: str = '') -> Dict[str, Any]:
        """
        Retrieve End of Day Timeseries data.
        """
        end_point = "/v1/listings/marketData/endOfDayHistory"
        query_string = {
            'scheme': scheme,
            'ids': ",".join(listings),
            'dateFrom': dateFrom,
            'dateTo': dateTo
        }
        return self._http_request(end_point, query_string)


In [27]:
findata = FinancialDataAPI('../SIXWebAPI/CH52991-hackathon7')

C:\Users\Lukas Volk\AppData\Local\Temp\ipykernel_28184\1408774091.py:22: DeprecationWarning: ssl.SSLContext() without protocol argument is deprecated.
  self.context = ssl.SSLContext()
C:\Users\Lukas Volk\AppData\Local\Temp\ipykernel_28184\1408774091.py:22: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self.context = ssl.SSLContext()


In [28]:
import os
import pandas as pd

file_path = os.path.join("../ESG/EUESGMANUFACTURER.csv")
try:
    # Load the CSV file
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"The file {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


In [29]:
unique_entries = df["ISIN"].unique()
unique_entries_list = list(unique_entries)

In [ ]:
sample1 = findata.instrumentBase("ISIN", unique_entries_list[:10])

In [ ]:
sample1["data"]["instruments"]

In [ ]:
import requests

def get_symbol_for_isin(isin):
    url = 'https://query1.finance.yahoo.com/v1/finance/search'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36',
    }

    params = dict(
        q=isin,
        quotesCount=1,
        newsCount=0,
        listsCount=0,
        quotesQueryId='tss_match_phrase_query'
    )

    resp = requests.get(url=url, headers=headers, params=params)
    data = resp.json()
    if 'quotes' in data and len(data['quotes']) > 0:
        return data['quotes'][0]['symbol']
    else:
        return None

In [ ]:
aggregate = {}

for i in unique_entries_list[:50]:
    aggregate.update({i: get_symbol_for_isin(i)})

In [ ]:
import csv

data = {key: value for key, value in aggregate.items() if value is not None}

filename = 'isin_to_ticker.csv'
with open(filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=data.keys())
    writer.writeheader()
    writer.writerow(data)

In [31]:
import yfinance as yf

def download_stock_data(ticker, isin):
    if not os.path.exists('history'):
        os.makedirs('history')

    if not os.path.exists('json'):
        os.makedirs('json')

    # Fetch historical data
    stock = yf.Ticker(ticker)
    
    info = stock.info
    
    # Define JSON file path
    json_filename = os.path.join('json', f"{isin}_info.json")
    
    # Write info to JSON file
    with open(json_filename, 'w') as jsonfile:
        json.dump(info, jsonfile, indent=4)
        print(f"Info saved to {json_filename}")
    
        hist_data = stock.history(period="max")
    
    # Define CSV file path
    csv_filename = os.path.join('history', f"{ticker}_historical_data.csv")
    
    # Save historical data to CSV
    hist_data.to_csv(csv_filename)
    print(f"Historical data saved to {csv_filename}")



for key, value in data.items():
    download_stock_data(value, key)

Info saved to json\CH0281016474_info.json
Info saved to json\CH0281016656_info.json
Info saved to json\CH0281017506_info.json
Info saved to json\CH0281018538_info.json
Info saved to json\CH0294813131_info.json
Info saved to json\CH0294813149_info.json
Info saved to json\CH0297178755_info.json
Info saved to json\CH0297178771_info.json
Info saved to json\CH0311188863_info.json
Info saved to json\CH0311189556_info.json
Info saved to json\CH0311189580_info.json
Info saved to json\CH0381682621_info.json
Info saved to json\CH0481373295_info.json
Info saved to json\CH0481373311_info.json
Info saved to json\CH0487357052_info.json
Info saved to json\CH0487357136_info.json
Info saved to json\CH0487357144_info.json
Info saved to json\CH0530164190_info.json
Info saved to json\CH0531747381_info.json
Info saved to json\CH0531753587_info.json
Info saved to json\CH0578494418_info.json
Info saved to json\CH0578494426_info.json
Info saved to json\CH0584288531_info.json
Info saved to json\CH0584288564_in